# Step 4: Imaging Integration
Overlay spatial clusters on tissue image for multi-modal interpretation.

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path

sc_path = Path('data/processed/human_breast_cancer_single_cell.h5ad')
spatial_path = Path('data/processed/human_breast_cancer_spatial.h5ad')
image_path = Path('data/raw/spatial/tissue_hires_image.png')

if not (sc_path.exists() and spatial_path.exists() and image_path.exists()):
    raise FileNotFoundError('Run Steps 1 and 3 first, and ensure tissue image exists')

adata = sc.read_h5ad(str(sc_path))
adata_spatial = sc.read_h5ad(str(spatial_path))
tissue_image = mpimg.imread(str(image_path))

if 'X_spatial' in adata_spatial.obsm:
    coords = adata_spatial.obsm['X_spatial']
elif 'spatial' in adata_spatial.obsm:
    coords = adata_spatial.obsm['spatial']
else:
    raise KeyError('No spatial coordinates found in adata_spatial.obsm')

x_coords = coords[:, 0]
y_coords = coords[:, 1]
img_height, img_width = tissue_image.shape[:2]
x_scaled = x_coords * img_width / np.max(x_coords)
y_scaled = y_coords * img_height / np.max(y_coords)

fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(tissue_image, alpha=0.7, extent=(0, img_width, img_height, 0))

if 'spatial_leiden' in adata_spatial.obs:
    cluster_colors = adata_spatial.obs['spatial_leiden'].astype('category').cat.codes
    scatter = ax.scatter(x_scaled, y_scaled, c=cluster_colors, s=20, alpha=0.8, cmap='tab20')
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Spatial Cluster')

ax.set_title('Human Breast Cancer: Spatial Transcriptomics + Tissue Image')
ax.set_xlabel('Spatial X Coordinate')
ax.set_ylabel('Spatial Y Coordinate')
plt.tight_layout()
plt.savefig('figures/Human Breast Cancer_multi_modal.png', dpi=300, bbox_inches='tight')
plt.close()

pd.DataFrame([{
    'dataset': 'Human Breast Cancer',
    'cells': adata.n_obs,
    'genes': adata.n_vars,
    'spots': adata_spatial.n_obs,
    'image_shape': str(tissue_image.shape),
    'integration_complete': True
}]).to_csv('results/imaging/Human Breast Cancer_integration_report.csv', index=False)

print('Step 4 complete')